# Library import

In [ ]:
import esm 
import pandas as pd
import torch
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler

import sys
sys.path.append("C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/")
import utils

In [9]:
# Load the pretrained model
pretrained_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()

# Print the structure to confirm the components
print(pretrained_model, end='\n\n')

ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=120, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((320,), eps=1e-05, elementwis

In [10]:
data_path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/data"
original_df = pd.read_csv(f'{data_path}/final_status_SPARE.csv')
original_df

,ProteinName_SPARE,Peptide_SPARE,Status_SPARE
0,sp|P02751|FINC_HUMAN,VDVIPVNLPGEHGQR,bon
1,sp|P02751|FINC_HUMAN,STTPDITGYR,bon
2,sp|P02751|FINC_HUMAN,SYTITGLQPGTDYK,bon
3,sp|P02751|FINC_HUMAN,IYLYTLNDNAR,bon
4,sp|P04114|APOB_HUMAN,TGISPLALIK,bon
...,...,...,...
150,sp|P02743|SAMP_HUMAN,VGEYSLYIGR,bon
151,sp|P04004|VTNC_HUMAN,GQYCYELDEK,mauvais
152,sp|P04004|VTNC_HUMAN,FEDGVLDPDYPR,bon
153,sp|P04004|VTNC_HUMAN,DWHGVPGQVDAAMAGR,bon


# Functions and loading data

In [ ]:
# Creating a dataframe with the sequences and labels
df = pd.DataFrame()
df["sequence"] = original_df["Peptide_SPARE"]
df["quantotypic"] = original_df.apply(lambda row: 0 if row['Status_SPARE'] =='bon' else 1, axis=1)

positive_df = df[df['quantotypic'] == 0]
negative_df = df[df['quantotypic'] == 1]

class_counts = df['quantotypic'].value_counts()
max_len = df['sequence'].str.len().max()
pos_weight = class_counts[0] / class_counts[1]
print(class_counts)
print(pos_weight)

quantotypic
0    117
1     38
Name: count, dtype: int64
3.0789473684210527


In [ ]:
def get_embeddings(sequences, model, batch_converter):
    """
    Computes sequence embeddings from a pretrained ESM model using the specified representation layer.

    Args:
        sequences (list of str): List of amino acid sequences.
        model (torch.nn.Module): Pretrained ESM model.
        batch_converter (callable): Function that converts (ID, sequence) pairs into tokenized inputs.

    Returns:
        list of np.ndarray: List of mean residue embeddings for each sequence.
    """
    embeddings = []

    with torch.no_grad():
        # Convert sequences to batch tokens using the ESM batch converter
        batch_labels, batch_strs, batch_tokens = batch_converter(
            [(f"seq{i}", seq) for i, seq in enumerate(sequences)]
        )

        # Get representations from a specific layer
        results = model(batch_tokens, repr_layers=[6])
        token_representations = results["representations"][6]

        for i, (_, seq) in enumerate(zip(batch_strs, sequences)):
            # Compute mean of residue representations (excluding [CLS] and padding)
            seq_embedding = token_representations[i, 1: len(seq) + 1].mean(0)
            embeddings.append(seq_embedding.cpu().numpy())

    return embeddings


# Extract embeddings for the sequences
sequences = df['sequence'].tolist()
batch_converter = alphabet.get_batch_converter()
embeddings = get_embeddings(sequences, pretrained_model, batch_converter)

df["embedding"] = embeddings
embeddings = np.array(embeddings)
print(embeddings.shape)

(155, 320)


# Random Forest

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
count = 1

# Trying with and without weighting
for weighting in [False, True]:
    temp_result_list = []
    # loop over the folds
    for fold, (train_idx, test_idx) in enumerate(kf.split(df['sequence'], df['quantotypic'])):
        # Split the data into training and test sets
        train_df = df.iloc[train_idx]
        test_df = df.iloc[test_idx]
        print(f"Fold: {fold}")
        print()

        # Load the pretrained model and batch converter
        pretrained_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
        batch_converter = alphabet.get_batch_converter()

        # Get the embeddings for the training and test sets
        train_sequences = train_df['sequence'].tolist()
        train_embeddings = get_embeddings(train_sequences, pretrained_model, batch_converter)
        X_train = np.vstack(train_embeddings)
        y_train = train_df['quantotypic'].values

        test_sequences = test_df['sequence'].tolist()
        test_embeddings = get_embeddings(test_sequences, pretrained_model, batch_converter)
        X_test = np.vstack(test_embeddings)
        y_test = test_df['quantotypic'].values

        # Param grid for RandomForestClassifier
        param_grid = {
            'max_depth': [None, 1, 3, 5, 6],
            'max_features': ['sqrt', 'log2', None], # None => max feature = n features
        }

        # if weighting is True, add class_weight to the classifier
        if weighting:
            rf = RandomForestClassifier(n_estimators=1000, random_state=42, class_weight='balanced')
        else:
            rf = RandomForestClassifier(n_estimators=1000, random_state=42)

        # Train the model using GridSearchCV
        grid_search_rf = GridSearchCV(rf, param_grid, cv=5, scoring='average_precision', verbose=4, n_jobs=-1)
        grid_search_rf.fit(X_train, y_train)

        print(grid_search_rf.best_params_)
        print(grid_search_rf.best_score_)

        # Retrieve the best parameters
        best_params = grid_search_rf.best_params_

        # Initialize a new RandomForestClassifier with the best parameters
        if weighting:
            best_rf = RandomForestClassifier(n_estimators=1000,
                **best_params,
                random_state=42,
                class_weight='balanced'
            )
        else:
            best_rf = RandomForestClassifier(n_estimators=1000,
                **best_params,
                random_state=42 
            )

        # Retrain the model on the entire training set
        best_rf.fit(X_train, y_train)

        # Predict on the test set
        y_pred = best_rf.predict(X_test)

        # Evaluate the model using the provided eval function
        accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

        metrics = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "roc_auc": roc_auc,
            "pr_auc": pr_auc
        }

        temp_result_list.append(metrics)

        # Print the results
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")
        print(f"PR AUC: {pr_auc:.4f}")

    accuracy_list = [result['accuracy'] for result in temp_result_list]
    precision_list = [result['precision'] for result in temp_result_list]
    recall_list = [result['recall'] for result in temp_result_list]
    f1_list = [result['f1'] for result in temp_result_list]
    roc_auc_list = [result['roc_auc'] for result in temp_result_list]
    pr_auc_list = [result['pr_auc'] for result in temp_result_list]

    metrics_summary = {
        "accuracy_mean": np.mean(accuracy_list),
        "accuracy_std": np.std(accuracy_list),
        "precision_mean": np.mean(precision_list),
        "precision_std": np.std(precision_list),
        "recall_mean": np.mean(recall_list),
        "recall_std": np.std(recall_list),
        "f1_mean": np.mean(f1_list),
        "f1_std": np.std(f1_list),
        "roc_auc_mean": np.mean(roc_auc_list),
        "roc_auc_std": np.std(roc_auc_list),
        "pr_auc_mean": np.mean(pr_auc_list),
        "pr_auc_std": np.std(pr_auc_list)
    }

    dict_save = {
        "weighting": weighting,
        "oversampling": False,
        "metrics_summary": metrics_summary
    }

    path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
    utils.write_into_json(dict_save, f"{path}/ESM_RF_experiment_{count}.json")
    count+=1

Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': None, 'max_features': 'log2'}
0.3874379517889027


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 3, 'max_features': None}
0.4241602681754116
Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.4792
PR AUC: 0.2258
Fold: 2

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': None, 'max_features': None}
0.42361515903543767
Accuracy: 0.6129
Precision: 0.2500
Recall: 0.2500
F1 Score: 0.2500
ROC AUC: 0.4946
PR AUC: 0.2560
Fold: 3

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 1, 'max_features': None}
0.4098092632341858


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581
Fold: 4

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 3, 'max_features': None}
0.40082817337461296
Accuracy: 0.7097
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.4783
PR AUC: 0.2581
Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 1, 'max_features': None}
0.45721282215090253
Accuracy: 0.5806
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.3750
PR AUC: 0.2258
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 5, 'max_features': None}
0.39727489501434277


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 2

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 3, 'max_features': None}
0.4175011246063877
Accuracy: 0.6452
Precision: 0.2857
Recall: 0.2500
F1 Score: 0.2667
ROC AUC: 0.5163
PR AUC: 0.2650
Fold: 3

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 1, 'max_features': 'sqrt'}
0.35164713182999624
Accuracy: 0.6129
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.4130
PR AUC: 0.2581
Fold: 4

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'max_depth': 1, 'max_features': None}
0.37556051396144274
Accuracy: 0.6129
Precision: 0.2500
Recall: 0.2500
F1 Score: 0.2500
ROC AUC: 0.4946
PR AUC: 0.2560


In [ ]:
# New loop with oversampling using ImbPipeline
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

temp_result_list = []
# loop over folds
for fold, (train_idx, test_idx) in enumerate(kf.split(df['sequence'], df['quantotypic'])):
    # Split the data into training and test sets
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]
    print(f"Fold: {fold}")
    print()

    # Load the pretrained model and batch converter
    pretrained_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
    batch_converter = alphabet.get_batch_converter()

    # Get the embeddings for the training and test sets
    train_sequences = train_df['sequence'].tolist()
    train_embeddings = get_embeddings(train_sequences, pretrained_model, batch_converter)
    X_train = np.vstack(train_embeddings)
    y_train = train_df['quantotypic'].values

    test_sequences = test_df['sequence'].tolist()
    test_embeddings = get_embeddings(test_sequences, pretrained_model, batch_converter)
    X_test = np.vstack(test_embeddings)
    y_test = test_df['quantotypic'].values

    # Param grid for RandomForestClassifier
    param_grid = {
    'classifier__max_depth': [None, 1, 3, 5, 6],
    'classifier__max_features': ['sqrt', 'log2', None],  # None = utiliser toutes les features
    }

    # Create the pipeline with oversampling and classifier
    pipeline = ImbPipeline([
        ('oversampler', RandomOverSampler(random_state=42)), 
        ('classifier', RandomForestClassifier(
            n_estimators=1000, 
            random_state=42
        ))
    ])

    # Training the model using GridSearchCV
    grid_search_rf = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,  # Validation croisée avec 5 splits
        scoring='average_precision',
        verbose=4,  # Affiche la progression
        n_jobs=-1  # Utilise tous les cœurs disponibles
    )

    grid_search_rf.fit(X_train, y_train)

    print(grid_search_rf.best_params_)
    print(grid_search_rf.best_score_)

    # Retrieve the best parameters
    best_params = grid_search_rf.best_params_

    best_rf = RandomForestClassifier(
        n_estimators=1000,  # Nombre d'arbres
        **{k.split('__')[1]: v for k, v in best_params.items()}, 
        random_state=42
    )

    # Retrain the model on the entire training set with oversampling
    oversampler = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)
    best_rf.fit(X_train, y_train)

    # Predict on the test set
    y_pred = best_rf.predict(X_test)

    # Evaluate the model using the provided eval function
    accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }

    temp_result_list.append(metrics)

    # Print the results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")

accuracy_list = [result['accuracy'] for result in temp_result_list]
precision_list = [result['precision'] for result in temp_result_list]
recall_list = [result['recall'] for result in temp_result_list]
f1_list = [result['f1'] for result in temp_result_list]
roc_auc_list = [result['roc_auc'] for result in temp_result_list]
pr_auc_list = [result['pr_auc'] for result in temp_result_list]

metrics_summary = {
    "accuracy_mean": np.mean(accuracy_list),
    "accuracy_std": np.std(accuracy_list),
    "precision_mean": np.mean(precision_list),
    "precision_std": np.std(precision_list),
    "recall_mean": np.mean(recall_list),
    "recall_std": np.std(recall_list),
    "f1_mean": np.mean(f1_list),
    "f1_std": np.std(f1_list),
    "roc_auc_mean": np.mean(roc_auc_list),
    "roc_auc_std": np.std(roc_auc_list),
    "pr_auc_mean": np.mean(pr_auc_list),
    "pr_auc_std": np.std(pr_auc_list)
}

dict_save = {
    "weighting": False,
    "oversampling": True,
    "metrics_summary": metrics_summary
}

path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
utils.write_into_json(dict_save, f"{path}/ESM_RF_experiment_3.json")

Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': 'log2'}
0.43270336632178735


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 5, 'classifier__max_features': None}
0.39321747346281505
Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.4792
PR AUC: 0.2258
Fold: 2

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 6, 'classifier__max_features': 'log2'}
0.42331495960752924
Accuracy: 0.7097
Precision: 0.4000
Recall: 0.2500
F1 Score: 0.3077
ROC AUC: 0.5598
PR AUC: 0.2935
Fold: 3

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 5, 'classifier__max_features': None}
0.36569407902612233
Accuracy: 0.6129
Precision: 0.1667
Recall: 0.1250
F1 Score: 0.1429
ROC AUC: 0.4538
PR AUC: 0.2466
Fold: 4

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': 'log2'}
0.3888393752

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# XGBoost

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
count = 1

# Trying with and without weighting
for weighting in [False, True]:
    temp_result_list = []

    # loop over the folds
    for fold, (train_idx, test_idx) in enumerate(kf.split(df['sequence'], df['quantotypic'])):
        # Split the data into training and test sets
        train_df = df.iloc[train_idx]
        test_df = df.iloc[test_idx]
        print(f"Fold: {fold}")
        print()

        # Load the pretrained model and batch converter
        pretrained_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
        batch_converter = alphabet.get_batch_converter()

        # Get the embeddings for the training and test sets
        train_sequences = train_df['sequence'].tolist()
        train_embeddings = get_embeddings(train_sequences, pretrained_model, batch_converter)
        X_train = np.vstack(train_embeddings)
        y_train = train_df['quantotypic'].values

        test_sequences = test_df['sequence'].tolist()
        test_embeddings = get_embeddings(test_sequences, pretrained_model, batch_converter)
        X_test = np.vstack(test_embeddings)
        y_test = test_df['quantotypic'].values

        # Param grid for XGBClassifier
        param_grid = {
            'max_depth': [None, 1, 3, 5, 6],              
            'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
        }

        # if weighting is True, add scale_pos_weight to the classifier
        if weighting:
            model = XGBClassifier(
                n_estimators=1000,
                scale_pos_weight=pos_weight, 
                random_state=42
            )
        else:
            model = XGBClassifier(
                n_estimators=1000,
                random_state=42
            )

        # Train the model using GridSearchCV
        grid_search = GridSearchCV(
            model, param_grid, cv=5, scoring="average_precision", verbose=4, n_jobs=-1
        )
        grid_search_rf.fit(X_train, y_train)

        print(grid_search_rf.best_params_)
        print(grid_search_rf.best_score_)

        # Retrieve the best parameters
        best_params = grid_search_rf.best_params_

        # Initialize a new XGBClassifier with the best parameters
        if weighting:
            best_model = XGBClassifier(
                n_estimators=1000,
                scale_pos_weight=pos_weight,
                random_state=42,
                **best_params
            )
        else:
            best_model = XGBClassifier(
                n_estimators=1000,
                random_state=42,
                **best_params
            )

        # Retrain the model on the entire training set
        best_rf.fit(X_train, y_train)

        # Predict on the test set
        y_pred = best_rf.predict(X_test)

        # Evaluate the model using the provided eval function
        accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

        metrics = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "roc_auc": roc_auc,
            "pr_auc": pr_auc
        }

        temp_result_list.append(metrics)

        # Print the results
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")
        print(f"PR AUC: {pr_auc:.4f}")

    accuracy_list = [result['accuracy'] for result in temp_result_list]
    precision_list = [result['precision'] for result in temp_result_list]
    recall_list = [result['recall'] for result in temp_result_list]
    f1_list = [result['f1'] for result in temp_result_list]
    roc_auc_list = [result['roc_auc'] for result in temp_result_list]
    pr_auc_list = [result['pr_auc'] for result in temp_result_list]

    metrics_summary = {
        "accuracy_mean": np.mean(accuracy_list),
        "accuracy_std": np.std(accuracy_list),
        "precision_mean": np.mean(precision_list),
        "precision_std": np.std(precision_list),
        "recall_mean": np.mean(recall_list),
        "recall_std": np.std(recall_list),
        "f1_mean": np.mean(f1_list),
        "f1_std": np.std(f1_list),
        "roc_auc_mean": np.mean(roc_auc_list),
        "roc_auc_std": np.std(roc_auc_list),
        "pr_auc_mean": np.mean(pr_auc_list),
        "pr_auc_std": np.std(pr_auc_list)
    }

    dict_save = {
        "weighting": weighting,
        "oversampling": False,
        "metrics_summary": metrics_summary
    }

    path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
    utils.write_into_json(dict_save, f"{path}/ESM_XGB_experiment_{count}.json")
    count+=1

Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': 'log2'}
0.43270336632178735


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 5, 'classifier__max_features': None}
0.39321747346281505


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 2

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 6, 'classifier__max_features': 'log2'}
0.42331495960752924


...

c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581
Fold: 3

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 5, 'classifier__max_features': None}
0.36569407902612233


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581
Fold: 4

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': 'log2'}
0.38883937527900375
Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# New loop with oversampling using ImbPipeline
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

temp_result_list = []
# loop over folds
for fold, (train_idx, test_idx) in enumerate(kf.split(df['sequence'], df['quantotypic'])):
    # Split the data into training and test sets
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]
    print(f"Fold: {fold}")
    print()

    # Load the pretrained model and batch converter
    pretrained_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
    batch_converter = alphabet.get_batch_converter()

    # Get the embeddings for the training and test sets
    train_sequences = train_df['sequence'].tolist()
    train_embeddings = get_embeddings(train_sequences, pretrained_model, batch_converter)
    X_train = np.vstack(train_embeddings)
    y_train = train_df['quantotypic'].values

    test_sequences = test_df['sequence'].tolist()
    test_embeddings = get_embeddings(test_sequences, pretrained_model, batch_converter)
    X_test = np.vstack(test_embeddings)
    y_test = test_df['quantotypic'].values

    # Param grid for XGBClassifier
    param_grid = {
        'classifier__max_depth': [None, 1, 3, 5, 6], 
        'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4], 
    }

    # Create the pipeline with oversampling and classifier
    pipeline = ImbPipeline([
        ('oversampler', RandomOverSampler(random_state=42)), 
        ('classifier', XGBClassifier(
            n_estimators=1000, 
            random_state=42
        ))
    ])

    # Training the model using GridSearchCV
    grid_search_xgb = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,  
        scoring='average_precision',
        verbose=4,
        n_jobs=-1
    )

    grid_search_rf.fit(X_train, y_train)

    print(grid_search_rf.best_params_)
    print(grid_search_rf.best_score_)

    # Retrieve the best parameters
    best_params = grid_search_rf.best_params_

    best_xgb = XGBClassifier(
        n_estimators=1000,
        random_state=42,
        **best_params 
    )

    # Retrain the model on the entire training set with oversampling
    oversampler = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)
    best_rf.fit(X_train, y_train)

    # Predict on the test set
    y_pred = best_rf.predict(X_test)

    # Evaluate the model using the provided eval function
    accuracy, precision, recall, f1, roc_auc, pr_auc = utils.eval(y_test, y_pred)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }

    temp_result_list.append(metrics)

    # Print the results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")

accuracy_list = [result['accuracy'] for result in temp_result_list]
precision_list = [result['precision'] for result in temp_result_list]
recall_list = [result['recall'] for result in temp_result_list]
f1_list = [result['f1'] for result in temp_result_list]
roc_auc_list = [result['roc_auc'] for result in temp_result_list]
pr_auc_list = [result['pr_auc'] for result in temp_result_list]

metrics_summary = {
    "accuracy_mean": np.mean(accuracy_list),
    "accuracy_std": np.std(accuracy_list),
    "precision_mean": np.mean(precision_list),
    "precision_std": np.std(precision_list),
    "recall_mean": np.mean(recall_list),
    "recall_std": np.std(recall_list),
    "f1_mean": np.mean(f1_list),
    "f1_std": np.std(f1_list),
    "roc_auc_mean": np.mean(roc_auc_list),
    "roc_auc_std": np.std(roc_auc_list),
    "pr_auc_mean": np.mean(pr_auc_list),
    "pr_auc_std": np.std(pr_auc_list)
}

dict_save = {
    "weighting": False,
    "oversampling": True,
    "metrics_summary": metrics_summary
}

path = "C:/Users/Walraff/OneDrive - Universite de Liege/Documents/Ulg/Master2/TFE/Results/ValidProtocol"
utils.write_into_json(dict_save, f"{path}/ESM_XGB_experiment_3.json")

Fold: 0

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': 'log2'}
0.43270336632178735


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 1

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 5, 'classifier__max_features': None}
0.39321747346281505


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7742
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2258
Fold: 2

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 6, 'classifier__max_features': 'log2'}
0.42331495960752924


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581
Fold: 3

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 5, 'classifier__max_features': None}
0.36569407902612233


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581
Fold: 4

Fitting 5 folds for each of 15 candidates, totalling 75 fits
{'classifier__max_depth': 1, 'classifier__max_features': 'log2'}
0.38883937527900375
Accuracy: 0.7419
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
ROC AUC: 0.5000
PR AUC: 0.2581


c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
